## Visualisation of LFP & Calcium traces

### Load packages

In [ ]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path
import IPython
import os
import copy
import json
from IPython.display import display
from ipyfilechooser import FileChooser
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer
from scipy.stats import zscore
from scipy.interpolate import interp1d
from itertools import groupby
import sys 
import pickle
import seaborn as sns
from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths
from scipy import stats
import matplotlib.colors as mcolors
import warnings
from scipy import interpolate
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer, CsvEpochSource, EpochEncoder,EpochViewer
from ephyviewer import InMemoryAnalogSignalSource
from ephyviewer import InMemorySpikeSource
from matplotlib import cm
from ast import literal_eval
import re
from scipy.ndimage import uniform_filter1d
from scipy.stats import zscore
warnings.filterwarnings("ignore")

#%matplotlib widget



def filterLFP(lfp, f_lowcut, f_hicut):
    range=f_hicut-f_lowcut
    fs = 1000
    nyq = 0.5 * fs
    N = 3 # Filtre order
    Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction
    b, a = signal.butter(N, Wn, 'band')
    filt= signal.filtfilt(b, a, lfp)
    # Parameter and computation of CWT
    w = 100. #window size
    freq = np.linspace(f_lowcut, f_hicut, range)
    widths = w*fs / (2*freq*np.pi)
    CWT = signal.cwt(filt, signal.morlet2, widths, w=w)
    # Projection calculation
    absCWT = np.absolute(CWT)
    zabsCWT = stats.zscore(absCWT, axis=None)
    proj_CWT = np.max(zabsCWT, axis = 0)/range
    return proj_CWT


In [ ]:
cd "C:/Users/Manip2/SCRIPTS/CodePythonAudrey/CodePythonAurelie/HayLabAnalysis/minian"

In [ ]:
from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    open_minian_mf,
    save_minian,
)

### Choose recording session

In [ ]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
    #dpath=dpath.parent
except:
    print("the path was not defined in store")
    dpath = "//10.69.168.1/crnldata/forgetting/"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Go inside the folder containing the LFP raw file</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

### Load signals

In [ ]:
folder_base = Path(dpath) 

print('DataFrame_rawdataDS.pkl file')
LFPfile = Path(f'{folder_base}/DataFrame_rawdataDS.pkl')
LFPs_df = pd.read_pickle(LFPfile)
samplerate = 1000 
numchannel = LFPs_df.shape[1]
rec_ch_list = LFPs_df.columns.values
# Load LFPs timestamps 
for file_pathTS in folder_base.parent.parent.glob('**/continuous/*/timeStampsDS.npy'):
    print('LFPs timestamps file found')
    LFPtimestamps = np.load(file_pathTS)  

#Load LFP coordinates 
notebook_path = Path("/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[-5:]))
Channels = Path(f'{notebook_path.parent}/_LFP_coordinates_of_all_mice.csv')
all_LFPcoordinates = pd.read_csv(Channels, index_col=0)

ScoringFile  = pd.read_csv(f'{folder_base}\Sleep_Scoring_6Stages_5sEpoch.csv')

#All = All0*0.195 # *0.195 to convert in µV

mappfile = open(folder_base.parent.parent.parent/ f'mappingsAB.pkl', 'rb')
mapping = pickle.load(mappfile)
mapping_sess = mapping['session']    

StampsFile = pd.read_excel(folder_base.parent/ f'SynchroFileCorrect.xlsx')
StartTime = list(StampsFile[0])[0]
minian_freq=list(StampsFile[0])[2]   

TSminiscope = pd.read_csv(list(folder_base.parent.glob('*V4_Miniscope/timeStamps.csv'))[0])
tsmini=TSminiscope['Time Stamp (ms)']

# Identify electrodes
mouse = []
pos_mouse = []
for mouse_name in all_LFPcoordinates.index:
    if mouse_name in LFPfile.__str__():
        mouse.append(mouse_name)
        pos_mouse.append(LFPfile.__str__().find(mouse_name)) 
mouse = [x for _, x in sorted(zip(pos_mouse, mouse))] # sort mouse in the same order as they appear in the path
mouse=mouse[0]
ID=0

all_LFPcoordinates = all_LFPcoordinates.astype(str)
for region in all_LFPcoordinates.loc[mouse].index:
    locals()[region] = []
    locals()[f'{region}_ch'] = []

combined = []
RecordedArea=[]
rec_ch_list_mouse = [value for value in rec_ch_list if 0+(ID*32) <= value <= 31+(ID*32)]
for rec_ch in rec_ch_list_mouse:
    for idx, LFPcoord_str in enumerate(all_LFPcoordinates.loc[mouse]):
        region = all_LFPcoordinates.loc[mouse].index[idx]
        if LFPcoord_str != 'nan':
            LFPcoord = LFPcoord_str.split('_')[:2] # only take into account the 2 first of electrode of that region 
            num_ch = np.where(str(rec_ch-(ID*32)) == np.array(LFPcoord))[0]
            if len(num_ch) > 0:
                region = all_LFPcoordinates.loc[mouse].index[idx]
                LFP = locals()[region]
                LFP = LFP-np.array(LFPs_df[(rec_ch)]) if len(LFP) > 0 else np.array(LFPs_df[(rec_ch)])
                locals()[region] = LFP
                locals()[f'{region}_ch'].append(rec_ch)
                break
            continue
for region in all_LFPcoordinates.loc[mouse].index:
    LFP = locals()[region]
    LFP_ch = locals()[f'{region}_ch']
    if len(LFP) > 0:
        combined = zscore(LFP[:,np.newaxis]) if len(combined) == 0 else np.append(combined, zscore(LFP[:,np.newaxis]), axis=1)
        RecordedArea.append(region) 


# Upscale Scoring
ScoringFile['label']= ScoringFile['label'].str.extract(r'(\d+)', expand=False)
SleepScoredTS=np.array(ScoringFile['label'])
scale_factor=samplerate/0.2  #cause scoring was done in 5 seconds bin, ie 0.2 Hz              
SleepScoredTS_upscaled0 = np.repeat(SleepScoredTS, scale_factor, axis=0)

if len(EMG) > len(SleepScoredTS_upscaled0):
    SleepScoredTS_upscaled = np.pad(SleepScoredTS_upscaled0, (0, (len(EMG) - len(SleepScoredTS_upscaled0))), constant_values=SleepScoredTS_upscaled0[-1])
elif len(EMG) < len(SleepScoredTS_upscaled0):
    SleepScoredTS_upscaled=SleepScoredTS_upscaled0[:len(EMG)]

print(round(LFPs_df.shape[0]/samplerate/60), 'min of recording')

In [ ]:
AllCalcium=pd.DataFrame()
AllSpike=pd.DataFrame()
Mapp={}
minian_folders = [f for f in folder_base.parents[0].rglob('minian') if f.is_dir()]

for minianpath in minian_folders: # for each minian folders found in this mouse 

    session=minianpath.parents[0].name if len(minianpath.parts)==12 else minianpath.parents[1].name.split("_")[-1]
    session_path=minianpath.parents[2] if len(minianpath.parts)==12 else minianpath.parents[1]
    expe_type=minianpath.parents[3].name if len(minianpath.parts)==12 else minianpath.parents[2].name

    session_date= minianpath.parents[2].name.split("_")[0] if len(minianpath.parts)==12 else minianpath.parents[1].name.split("_")[0]
    session_time= minianpath.parents[2].name.split("_")[1] if len(minianpath.parts)==12 else minianpath.parents[1].name.split("_")[1]

    drug='CGP' if expe_type == 'postCGP' else 'baseline'

    minian_ds = open_minian(minianpath)
    CalciumSub1 = minian_ds['C'] # calcium traces 
    SpikeSub1 = minian_ds['S'] # estimated spikes deconvolved activity
    Mapp[session] = minian_ds['A'] # estimated spikes deconvolved activity

    CalciumSub = pd.DataFrame(CalciumSub1, index=CalciumSub1['unit_id'])
    recdur = CalciumSub.dropna(axis=1, how='all').shape[1] #cause Nans were added to match the different subsessions video sizes
    CalciumSub = CalciumSub.dropna(axis=1, how='all') #cause Nans were added to match the different number of units detected per subsessions 
    CalciumSub = CalciumSub.dropna(axis=0, how='all') #cause Nans were added to match the different number of units detected per subsessions 
    
    SpikeSub = pd.DataFrame(SpikeSub1, index=CalciumSub1['unit_id'])
    SpikeSub = SpikeSub.dropna(axis=0, how='all') #cause Nans were added to match the different number of units detected per subsessions 
    SpikeSub = SpikeSub.dropna(axis=1, how='all') #cause Nans were added to match the different subsessions video sizes
    
    indexMappList=mapping_sess[session]
    kept_uniq_unit_List=[]
    for unit in CalciumSub.index:
        indexMapp = np.where(indexMappList == unit)[0]
        kept_uniq_unit_List.append(str(indexMapp))

    with open(minianpath / f'TodropFileAB.json', 'r') as f:
        unit_to_drop = json.load(f)
        TodropFile  = unit_to_drop

    Mapp[session]=Mapp[session].drop_sel(unit_id=unit_to_drop)

    for u in unit_to_drop: 
        CalciumSub=CalciumSub.drop(index=u) if u in CalciumSub.index else CalciumSub
        SpikeSub=SpikeSub.drop(index=u) if u in SpikeSub.index else SpikeSub

    kept_uniq_unit_List=[]
    for unit in CalciumSub.index:
        indexMapp = np.where(indexMappList == unit)[0]
        kept_uniq_unit_List.append(str(indexMapp))

    CalciumSub.index=[f"{mouse}{str(i).replace('[','').replace(']','')}" for i in kept_uniq_unit_List]
    
    kept_uniq_unit_List=[]
    for unit in SpikeSub.index:
        indexMapp = np.where(indexMappList == unit)[0]
        kept_uniq_unit_List.append(str(indexMapp))

    SpikeSub.index=[f"{mouse}{str(i).replace('[','').replace(']','')}" for i in kept_uniq_unit_List]

    CalciumSub=CalciumSub[CalciumSub.index!=mouse] #remove nr not in the cross registration
    SpikeSub=SpikeSub[SpikeSub.index!=mouse]

    print(CalciumSub.shape)

    AllCalcium=pd.concat([AllCalcium, CalciumSub.T], axis=0, ignore_index=True) #if CalciumSub.shape[0] else pd.concat([AllCalcium, pd.DataFrame(np.full((1, CalciumSub1.shape[1]), np.nan))], ignore_index=True)
    AllSpike=pd.concat([AllSpike, SpikeSub.T], axis=0, ignore_index=True)# if CalciumSub.shape[0] else pd.concat([AllSpike, pd.DataFrame(np.full((1, CalciumSub1.shape[1]), np.nan))], ignore_index=True)

unaligned_Calcium=AllCalcium.T.copy()
unaligned_Spike=AllSpike.T.copy()
print(np.shape(unaligned_Calcium), 'unaligned')

# Realigned to the real minian frequency and the timestamps 
timestamps =  np.array(tsmini)/samplerate
x_values = unaligned_Calcium  # Each row is a Cell, each column corresponds to a timestamp
sample_rate = minian_freq  # Hz
new_timestamps= np.arange(timestamps[0], timestamps[-1], 1/sample_rate)
Calcium = pd.DataFrame(index=x_values.index, columns=new_timestamps)
for cell in x_values.index:
    interpolator = interpolate.interp1d(timestamps, x_values.loc[cell], kind='linear')
    Calcium.loc[cell] = interpolator(new_timestamps)

x_values = unaligned_Spike  # Each row is a cell, each column corresponds to a timestamp
new_timestamps= np.arange(timestamps[0], timestamps[-1], 1/sample_rate)
Spike = pd.DataFrame(index=x_values.index, columns=new_timestamps)
for cell in x_values.index:
    interpolator = interpolate.interp1d(timestamps, x_values.loc[cell], kind='linear')
    Spike.loc[cell] = interpolator(new_timestamps)
print(np.shape(Calcium), 'aligned')

In [ ]:
# Load Cluster ID per units
try: 
    p='//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/Analysed2025_AB/_baseline_analysis/VigSt_2025-05-03_10_01_32/VigStates_Global_cluster.xlsx'
    data=pd.read_excel(p)
    data = data.drop_duplicates(subset=['Unit_ID']) 
    d=pd.DataFrame()
    d['Unit_ID']=data['Unit_ID']
    d['ClusterHDBSCAN']=data['ClusterHDBSCAN']
    bool=[i in Calcium.index for i in d['Unit_ID']] # find common units (the ones from that mouse and that session)
    d['Unit_ID'][bool]
    db=d[bool]
    db.index=db['Unit_ID']
    dba=db['ClusterHDBSCAN']
    Cluster=np.array([dba[i] for i in Calcium.index])
    df=pd.DataFrame(data={'Cluster':Cluster}, index=Calcium.index)
    df=df.sort_values(by='Cluster')
    name=[f'{df["Cluster"][i]}_{i}' for i in df.index]
    Calcium_sorted = Calcium.reindex(df.index)
    Spike_sorted = Spike.reindex(df.index)
    clusterID=1    
except: 
    clusterID=0

In [ ]:
# Load detected Cell assemblies
def extract_numbers(s):
    matches = re.findall(r'\[([^\[\]]+)\]', s)  # only non-empty brackets
    numbers = []
    for match in matches:
        nums = [float(x) for x in match.strip().split()]
        numbers.extend(nums)
    return numbers
try: 
    p='//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/Analysed2025_AB/_baseline_analysis/VigSt_2025-04-21_14_04_43_noCorr_1000iter/CellAssembly_Global_cluster_ID.xlsx'
    cellass_df=pd.read_excel(p)
    cellass_df = cellass_df[cellass_df['mouse']== mouse]
    cellass_df = cellass_df[cellass_df['Session_Date']== session_date]
    cellass_df = cellass_df[cellass_df['Session_Time']== session_time]
    CellA = cellass_df.groupby(['Assembly_ID', 'Cells_in_Assembly' ], as_index=False).agg({'EventTime': ' '.join})
    CellA['EventTime']= CellA['EventTime'].apply(extract_numbers)
    eventCellA = CellA['EventTime']
    cellAssemblyID=1     
except: 
    cellAssemblyID=0

In [ ]:
# Load detected oscillations timestamps
# SWR     
SWSdetection=folder_base / f'SWR_detection.csv'
SWR_prop = pd.read_csv(SWSdetection, index_col=0)  
SWR_prop['toKeep'] = SWR_prop['toKeep'].astype(str)
SWR_prop  = SWR_prop[SWR_prop['toKeep'].isin(['VRAI', 'True'])]   
SWR_start = np.transpose((SWR_prop['start time']).astype(int))
SWR_end = np.transpose((SWR_prop['end time']).astype(int))

# Spindles
Spdldetection=folder_base / f'SpindlesS1&PFC_detection.csv'
Spdl_prop = pd.read_csv(Spdldetection, index_col=0)  
Spdl_prop['toKeep'] = Spdl_prop['toKeep'].astype(str)
Spdl_prop  = Spdl_prop[Spdl_prop['toKeep'].isin(['VRAI', 'True'])]   

S1Spdl_prop= Spdl_prop[Spdl_prop['CTX']=='S1']
S1spdl_start = np.transpose(S1Spdl_prop['start time'].astype(int))
S1spdl_end = np.transpose(S1Spdl_prop['end time'].astype(int))

PFCSpdl_prop= Spdl_prop[Spdl_prop['CTX']=='PFC']
PFCspdl_start = np.transpose(PFCSpdl_prop['start time'].astype(int))
PFCspdl_end = np.transpose(PFCSpdl_prop['end time'].astype(int))

GlobalSpdl_prop= Spdl_prop[Spdl_prop['CTX']=='S1PFC']
GBLspdl_start = np.transpose(GlobalSpdl_prop['start time'].astype(int))
GBLspdl_end = np.transpose(GlobalSpdl_prop['end time'].astype(int))

print(f'{len(SWR_end)} SWR in CA1, {len(S1spdl_end)} Spdl in S1, {len(PFCspdl_end)} Spdl in PFC, {len(GBLspdl_end)} Global Spdl')

### Visualization according to cluster ID & cell assemblies

In [ ]:
labels = ['AW_1', 'QW_2', 'NREM_3', 'IS_4',  'REM_5', 'undefined_6'] # Active wake, Quiet wake, NREM sleep, Intermediate sleep, REM sleep, undefined states
epoch_dur = 5 # define epoch duration in sec
winlen = 10

app = mkQApp()
win = MainViewer(debug=False, show_auto_scale=True)
t_start=0.


# LFPs
try: 
    scatter_indexes = {0: PFCspdl_start, 1: PFCspdl_end, 2: S1spdl_start, 3: S1spdl_end, 4: GBLspdl_start, 5: GBLspdl_end, 6: SWR_start, 7: SWR_end}
    scatter_channels = {0: [2], 1: [2], 2: [1], 3: [1], 4: [1,2], 5: [1,2], 6: [3], 7: [3]}
except: 
    scatter_indexes={}
    scatter_channels={}
source = AnalogSignalSourceWithScatter(combined, samplerate, t_start, scatter_indexes, scatter_channels, scatter_colors= {0: '#FFFFFF', 1: '#222222', 2: '#FFFFFF', 3: '#222222', 4: '#BBBBBB', 5: '#555555', 6: '#FFFFFF', 7: '#222222'}, channel_names=RecordedArea)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
for idx, ch in enumerate(RecordedArea): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx] #FF0000 red, #00FF00 green, and #0000FF blue
view1.auto_scale()


# Calcium traces 
if clusterID: 
    view2 = TraceViewer.from_numpy((np.array(Calcium_sorted.T)), float(minian_freq), round(StartTime,2), 'Calcium Traces',channel_names=name)
    colormap = ['#ffcd69', '#69cfff', '#ff69cd','#FFFFFF']
    clus = np.array([s.split('_')[0] for s in name])#.astype(int)
    for idx, ch in enumerate(clus): 
        ch = 3 if ch=='nan' else int(float(ch))
        view2.by_channel_params[f'ch{idx}', 'color'] = colormap[ch] #FF0000 red, #00FF00 green, and #0000FF blue
else: 
    view2 = TraceViewer.from_numpy((np.array(Calcium.T)), float(minian_freq), round(StartTime,2), 'Calcium Traces',channel_names=Calcium.index)

view2.params['scale_mode'] = 'same_for_all'
view2.auto_scale()
view2.params['display_labels'] = True


# Spike traces or Cell assemblies event
if cellAssemblyID:
    all_assemblies = []
    for c,row in enumerate(eventCellA):
        all_assemblies.append({ 'time':np.array(row), 'name':list(CellA['Cells_in_Assembly'].unique())[c] })
    source = InMemorySpikeSource(all_spikes=all_assemblies)
    view3 = ephyviewer.SpikeTrainViewer(source=source, name='Cell assemblies event')    
    if len(all_assemblies)>0:
        cmap = plt.cm.get_cmap('Spectral', len(all_assemblies))
        colors = cmap(np.linspace(0, 1, len(all_assemblies)))
        hex_colors = [mcolors.to_hex(color[:3]) for color in colors]
        for idx, ch in enumerate(np.arange(len(all_assemblies))): 
            view3.by_channel_params[f'ch{idx}', 'color'] = hex_colors[idx] #FF0000 red, #00FF00 green, and #0000FF blue        
elif clusterID:
    all_spikes = []
    for c,row in enumerate(np.array(Spike_sorted)):
        peaks, _ = find_peaks(row)#, height=np.nanstd(row)/5)
        all_spikes.append({ 'time':(peaks/minian_freq)+round(StartTime,2), 'name':Calcium.index[c] })
    source = InMemorySpikeSource(all_spikes=all_spikes)
    view3 = ephyviewer.SpikeTrainViewer(source=source, name='Spikes')
    colormap = ['#ffcd69', '#69cfff', '#ff69cd']
    clus = np.array([s.split('_')[0] for s in name]).astype(float)
    for idx, ch in enumerate(clus.astype(int)): 
        view3.by_channel_params[f'ch{idx}', 'color'] = colormap[ch] if ch !=-2147483648 else  '#ffffff' #FF0000 red, #00FF00 green, and #0000FF blue
else: 
    all_spikes = []
    for c,row in enumerate(np.array(Spike)):
        peaks, _ = find_peaks(row)#, height=np.nanstd(row)/5)
        all_spikes.append({ 'time':(peaks/minian_freq)+round(StartTime,2), 'name':Calcium.index[c] })
    source = InMemorySpikeSource(all_spikes=all_spikes)
    view3 = ephyviewer.SpikeTrainViewer(source=source, name='Spikes')
view3.auto_scale()
view3.params['display_labels'] = True


# Scoring 
SleepScoring_filename = f'{dpath}/Sleep_Scoring_{len(labels)}Stages_{epoch_dur}sEpoch.csv'
source_epoch = CsvEpochSource(SleepScoring_filename, labels)
view4 = EpochEncoder(source=source_epoch, name='Sleep Scoring')

view4.params['xsize'] = winlen
view4.params['new_epoch_step'] = epoch_dur
view4.by_label_params['label0', 'color'] = '#ffcd69' #AW
view4.by_label_params['label1', 'color'] = '#69ffe6' #QW
view4.by_label_params['label2', 'color'] = '#69cfff' #NREM
view4.by_label_params['label3', 'color'] = '#cd69ff' #IS
view4.by_label_params['label4', 'color'] = '#ff69cd' #REM
view4.by_label_params['label5', 'color'] = '#8f8f8f' #undefined
view4.controls.hide()


#add them to mainwindow
win.add_view(view1)
win.add_view(view4)
win.add_view(view2)
win.add_view(view3)


#Run
win.show()
app.exec_()

### Visualization with FFT

In [ ]:
labels = ['AW_1', 'QW_2', 'NREM_3', 'IS_4',  'REM_5', 'undefined_6'] # Active wake, Quiet wake, NREM sleep, Intermediate sleep, REM sleep, undefined states
epoch_dur = 5 # define epoch duration in sec
winlen = 10

app = mkQApp()
win = MainViewer(debug=True, show_auto_scale=True)

# LFPs
try: 
    scatter_indexes = {0: PFCspdl_start, 1: PFCspdl_end, 2: S1spdl_start, 3: S1spdl_end, 4: GBLspdl_start, 5: GBLspdl_end, 6: SWR_start, 7: SWR_end}
    scatter_channels = {0: [2], 1: [2], 2: [1], 3: [1], 4: [1,2], 5: [1,2], 6: [3], 7: [3]}
except: 
    scatter_indexes={}
    scatter_channels={}

#Create the main window that can contain several viewers
win = MainViewer(debug=False, show_auto_scale=True)
t_start=0.
#create a viewer for signal
source = AnalogSignalSourceWithScatter(combined, samplerate, t_start, scatter_indexes, scatter_channels, scatter_colors= {0: '#FFFFFF', 1: '#222222', 2: '#FFFFFF', 3: '#222222', 4: '#BBBBBB', 5: '#555555', 6: '#FFFFFF', 7: '#222222'}, channel_names=RecordedArea)
view1 = TraceViewer(source=source)

view1.params['xsize']= winlen
view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
colormap = np.insert(['#88FF88', '#8888FF', '#FF8888']* 10, 0, '#FFFFFF')
for idx, ch in enumerate(RecordedArea): 
    view1.by_channel_params[f'ch{idx}', 'color'] = colormap[idx] #FF0000 red, #00FF00 green, and #0000FF blue
view1.auto_scale()


# Calcium traces 
if clusterID: 
    view2 = TraceViewer.from_numpy((np.array(Calcium_sorted.T)), float(minian_freq), round(StartTime,2), 'Calcium Traces',channel_names=name)
    colormap = ['#ffcd69', '#69cfff', '#ff69cd']
    clus = np.array([s.split('_')[0] for s in name]).astype(float)
    for idx, ch in enumerate(clus): 
        view2.by_channel_params[f'ch{idx}', 'color'] = colormap[ch] #FF0000 red, #00FF00 green, and #0000FF blue
else: 
    view2 = TraceViewer.from_numpy((np.array(Calcium.T)), float(minian_freq), round(StartTime,2), 'Calcium Traces',channel_names=Calcium.index)

view2.params['scale_mode'] = 'same_for_all'
view2.auto_scale()
view2.params['display_labels'] = True


# FFT
view3 = TimeFreqViewer(source=source, name='FFT')

view3.params['show_axis'] = True
view3.params['timefreq', 'f_start'] = 10
view3.params['timefreq', 'f_stop'] = 18
view3.params['timefreq', 'deltafreq'] = 1 #interval in Hz
view3.params['xsize'] = winlen
for idx, ch in enumerate(RecordedArea):
    if ch == 'EMG': 
        view3.by_channel_params[f'ch{idx}', 'visible'] = False
    else:        
        view3.by_channel_params[f'ch{idx}', 'clim'] = 1
        view3.by_channel_params[f'ch{idx}', 'visible'] = True


# Scoring 
SleepScoring_filename = f'{dpath}/Sleep_Scoring_{len(labels)}Stages_{epoch_dur}sEpoch.csv'
source_epoch = CsvEpochSource(SleepScoring_filename, labels)
view4 = EpochEncoder(source=source_epoch, name='Sleep Scoring')

view4.params['xsize'] = winlen
view4.params['new_epoch_step'] = epoch_dur
view4.by_label_params['label0', 'color'] = '#ffcd69' #AW
view4.by_label_params['label1', 'color'] = '#69ffe6' #QW
view4.by_label_params['label2', 'color'] = '#69cfff' #NREM
view4.by_label_params['label3', 'color'] = '#cd69ff' #IS
view4.by_label_params['label4', 'color'] = '#ff69cd' #REM
view4.by_label_params['label5', 'color'] = '#8f8f8f' #undefined
view4.controls.hide()


#add them to mainwindow

win.add_view(view1)
win.add_view(view3)
win.add_view(view4)
win.add_view(view2)

#Run

win.show()
app.exec_()

## Create plots for paper

Choose recording extract

In [ ]:
beg=10 #in sec
fin=beg+(60*15) # 10 min recording

begCa=round((beg-StartTime)*minian_freq)
finCa=round((fin-StartTime)*minian_freq)

Plot raw calcium traces

In [ ]:
# Plot Ca traces all together
plt.close()
#A=Calcium[Calcium.columns[:17000]]
A=copy.deepcopy(Calcium[Calcium.columns[begCa:finCa]])
A = A.iloc[::] if 'L2_3_mice' in str(folder_base) else A#.iloc[1::2] #A.iloc[3::3]
A = A.dropna()
#A= A.apply(zscore, axis=1)
A = A.div(A.max(axis=1), axis=0)
num_lines = A.shape[0]
# Create a colormap
cmap = plt.get_cmap('viridis')
cmap = sns.light_palette("black", as_cmap=True) if 'L2_3_mice' in str(folder_base) else sns.light_palette("#008B8B", as_cmap=True)
# Normalize line indices to colormap range
colors = cmap(np.linspace(0,.8, num_lines)) if 'L2_3_mice' in str(folder_base) else cmap(np.linspace(0.4, 1, num_lines))
# Plot each line with its corresponding color
plt.figure(figsize=(6, 1))
for i in range(num_lines):
    line_data = A.iloc[i]
    plt.plot(A.columns, line_data, color=colors[i], linewidth=1)  # Change '1.5' to adjust the gap
plt.axis('off')  # Hides the axis lines and labels
plt.xticks([])   # Hides the x-axis ticks
plt.yticks([]) 
plt.tight_layout()
#plt.savefig(f'C:/Users/Manip2/Documents/ElifePaper/Rawdata/Extract{mouse}_{folder_base.parts[-2]}_line_2Catraces_{beg}to{fin}s.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True) 
plt.show()

In [ ]:
# Plot Ca traces aligned
plt.close()
#A=Calcium[Calcium.columns[:17000]]
A=copy.deepcopy(Calcium_sorted[Calcium_sorted.columns[begCa:finCa]])
A=copy.deepcopy(Calcium[Calcium.columns[begCa:finCa]])
A = A.loc[::] if 'L2_3_mice' in str(folder_base) else A#.iloc[1::2] #A.iloc[3::3]
A = A.dropna()
#A= A.apply(zscore, axis=1)
A = A.div(A.max(axis=1), axis=0)*2
A = A.dropna()
num_lines = A.shape[0]
# Create a colormap
cmap = plt.get_cmap('viridis')
cmap = sns.light_palette("black", as_cmap=True) if 'L2_3_mice' in str(folder_base) else sns.light_palette("#008B8B", as_cmap=True)
# Normalize line indices to colormap range
colors = cmap(np.linspace(0.2, .9, num_lines)) if 'L2_3_mice' in str(folder_base) else cmap(np.linspace(0.3, 1, num_lines))
# Plot each line with its corresponding color
plt.figure(figsize=(6, 5))
#plt.text(A.columns[0] - 100,0, f'#1', color=colors[0], 
#            ha='right', va='center', fontsize=6)  
for i in range(num_lines):
    line_data = A.iloc[i] - i * 1
    plt.plot(A.columns, line_data, color=colors[i], linewidth=1)  # Change '1.5' to adjust the gap
    # Add a label slightly to the left of each line
    plt.text(A.columns[0] - 100, min(line_data), f'#{A.index[i]}', color=colors[i], ha='right', va='center', fontsize=6)  
    plt.text(A.columns[0] - 50, min(line_data), f'#{i+1}', color=colors[i], ha='right', va='center', fontsize=6)  
plt.axis('off')  # Hides the axis lines and labels
plt.xticks([])   # Hides the x-axis ticks
plt.yticks([])  
plt.tight_layout()
plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/Figure1_revised/Extract{mouse}_{folder_base.parts[-2]}_line_2Catraces_{beg}to{fin}s.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()

In [ ]:
# Plot Ca traces according to Cluster ID
plt.close()
#A=Calcium[Calcium.columns[:17000]]
A=copy.deepcopy(Calcium_sorted[Calcium_sorted.columns[begCa:finCa]])
A = A.iloc[::] if 'L2_3_mice' in str(folder_base) else A.iloc[3::]
namebis=name[3::]
#first_half_selected = A.iloc[::]
#second_half = A.iloc[:]
#A = pd.concat([first_half_selected, second_half])
A = A.dropna()
#A= A.apply(zscore, axis=1)
A = A.div(A.max(axis=1), axis=0)*4
num_lines = A.shape[0]
# Create a colormap
cmap = plt.get_cmap('viridis')
cmap = sns.light_palette("black", as_cmap=True) if 'L2_3_mice' in str(folder_base) else sns.light_palette("#008B8B", as_cmap=True)
# Normalize line indices to colormap range
colors = cmap(np.linspace(0.4, 1, num_lines)) if 'L2_3_mice' in str(folder_base) else cmap(np.linspace(0.4, 1, num_lines))
# Plot each line with its corresponding color
plt.figure(figsize=(10, 4))
for i in range(num_lines):
    line_data = A.iloc[i] - i * 1.5
    plt.plot(A.columns, line_data, color=colors[i])  # Change '1.5' to adjust the gap
    label = namebis[i]
    # Add a label slightly to the left of each line
    plt.text(A.columns[0] - 100, min(line_data), f'#{i+1}', color=colors[i], 
            ha='right', va='center', fontsize=8)   
    plt.text(A.columns[-1] + 200, min(line_data), label, color=colors[i], 
            ha='left', va='center', fontsize=8)     
plt.axis('off')  # Hides the axis lines and labels
plt.xticks([])   # Hides the x-axis ticks
plt.yticks([])  
plt.tight_layout()
plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/Figure2_revised/Extract{mouse}_{folder_base.parts[-2]}_line_2Catraces_SI{beg}to{fin}s.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()

In [ ]:
# Plot Hynogram
colors = ['black', 'grey','darkgrey', 'lightgrey', 'white', 'red']
cmap = mcolors.ListedColormap(colors)
bounds = [1, 2, 3, 4, 5, 6, 7]
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=len(colors))

# Extract and prepare DataFrame
df = pd.DataFrame(SleepScoredTS_upscaled[beg*samplerate:fin*samplerate], columns=['Value'])
df['Value'] = df['Value'].astype(int)  # ensure values are ints for fmt="d"

# Plot
plt.figure(figsize=(8, 0.25))
heatmap = sns.heatmap(df['Value'].values.reshape(1, -1), cmap=cmap, norm=norm, fmt="d")

# Customize appearance
cbar = heatmap.collections[0].colorbar
cbar.remove()
plt.xticks([])
plt.yticks([])
plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/Figure2_revised/Extract{mouse}_{folder_base.parts[-2]}_Hypno_{beg}to{fin}s.png', format='png', bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()


In [ ]:
Mapp.keys()

In [ ]:
#Cell Mapp
M=Mapp['session3_2']
values = M['unit_id']
idloc = M.idxmax("unit_id")
Hmax = M.idxmax("height")
Hmax2 = Hmax.max("width")

Wmax = M.idxmax("width")
Wmax2 = Wmax.max("height")
coord1 = Wmax2.to_series()
coord2 = Hmax2.to_series()

a = pd.concat([coord1,coord2], axis=1)

plt.imshow(M.max("unit_id").compute().astype(np.float32), cmap='viridis', )
# Plot text
for n,val in enumerate(values):
    position = np.where(a.index == val.values)[0]
    position = position[0]
    #plt.text( a.iloc[position, 0],  a.iloc[position, 1], str(val.values), color='white', ha='center', va='center', fontsize=8)
    plt.text( a.iloc[position, 0],  a.iloc[position, 1], str(n+1), color='white', ha='center', va='center', fontsize=8)

plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/Figure1_revised/Extract{mouse}_{folder_base.parts[-2]}_CellMapp_{beg}to{fin}s.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True)


Plot correlation matrix

In [ ]:
# Correlation matrix all vig states
plt.close()
VMAX=.8
VMIN=-0.1
CaCorrMatrix=[]
CaCorrMatrix = pd.DataFrame(columns=[f'#{i}' for i in range(len(A))], index=[f'#{i}' for i in range(len(A))])
for unit in range(len(A)): 
    Carray_unit =A.iloc[unit]
    for unit2 in range(len(A)):
        Carray_unit2 =A.iloc[unit2]
        indexMapp =unit
        indexMapp2 = unit2                            
        corr_matrix = np.corrcoef(Carray_unit.astype(float), Carray_unit2.astype(float))
        CaCorrMatrix[f'#{indexMapp}'][f'#{indexMapp2}']={1: 0.99999, -1: -0.99999}.get(np.round(corr_matrix[0, 1],5), np.round(corr_matrix[0, 1],5))
    
df = CaCorrMatrix.apply(pd.to_numeric, errors='coerce')
plt.figure(figsize=(2.5, 2.5))
ax = sns.heatmap(df, square=True, vmin=VMIN , vmax=VMAX , cmap='viridis',
                                  cbar_kws={'shrink': 0.8, 'aspect': 20})
# Set only the first and last ticks for x and y axes
# Get the number of ticks
x_ticks = ax.get_xticks()
y_ticks = ax.get_yticks()
ax.set_xticks([x_ticks[0], x_ticks[-1]])
ax.set_yticks([y_ticks[0], y_ticks[-1]])

# Optionally, set the labels for these ticks if needed
ax.set_xticklabels([f'#{int(x_ticks[0])+1}', f'#{int(x_ticks[-1])+1}'],size=6)
ax.set_yticklabels([f'#{int(y_ticks[0])+1}', f'#{int(y_ticks[-1])+1}'],size=6)

# Optionally, add labels for the axes if needed
ax.set_xlabel('L2/3  neurons', labelpad=-2,size=8) if 'L2_3_mice' in str(folder_base) else ax.set_xlabel('L1  neurons', labelpad=-2,size=8)
ax.set_ylabel('L2/3 neurons', labelpad=-8,size=8) if 'L2_3_mice' in str(folder_base) else ax.set_ylabel('L1 neurons', labelpad=-8,size=8)
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=6)  # Adjust the font size here
cbar.set_label('Correlation coefficient (r)', fontsize=8, font='Arial', rotation=-90, labelpad=10)
ax.xaxis.set_ticks_position('top')  # Move ticks to the top
ax.xaxis.set_label_position('top')  # Move label to the top
plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/Figure2_revised/Extract{mouse}_{folder_base.parts[-2]}_2line_{beg}to{fin}s_heatmap.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()

In [ ]:
# Correlation matrix per vig states
plt.close()

# Downscale Scoring to Calcium rec

SleepScoredTS_upscaledCa=SleepScoredTS_upscaled[round(StartTime*samplerate):round((StartTime + (len(Calcium_sorted.T)/minian_freq))*samplerate)]
SleepScoredTS_upscaledCa=SleepScoredTS_upscaled[round(StartTime*samplerate):round((StartTime + (len(Calcium_sorted.T)/minian_freq))*samplerate)]
indices = np.linspace(0, len(SleepScoredTS_upscaledCa) - 1, len(SleepScoredTS_upscaledCa))
new_indices = np.linspace(0, len(SleepScoredTS_upscaledCa) - 1, len(Calcium_sorted.T))
interpolated_func = interp1d(indices, SleepScoredTS_upscaledCa, kind='zero')
SleepScoredTS_upscaledCa2 = interpolated_func(new_indices)
SleepScoredTS_upscaledCa2 = np.round(SleepScoredTS_upscaledCa2 * 2) / 2


VMAX=.8
CaCorrAW=[]
CaCorrQW=[]
CaCorrNREM=[]
CaCorrREM=[]
#mapp= {1: 'AW',2: 'QW', 3: 'NREM', 4: 'IS', 5: 'REM', 6: 'undefined'}
mapp= {1: 'AW', 2: 'QW', 3: 'NREM', 5: 'REM'}
fig, axes = plt.subplots(1, len(mapp), figsize=(10, 4))

for p,m in enumerate(mapp):
    # Correlation between each neurons according to vigilance states 
    Bool = (SleepScoredTS_upscaledCa2[begCa:finCa] == m)
    Ac = np.array(A.copy())
    Ac = Ac[:, Bool]
    CaCorrMatrix=[]
    CaCorrMatrix = pd.DataFrame(columns=[i for i in range(len(A))], index=[i for i in range(len(A))])

    for unit in range(len(A)): 
        Carray_unit =Ac[unit,:]
        for unit2 in range(len(A)): 
            Carray_unit2 =Ac[unit2,:]
            indexMapp = unit
            indexMapp2 = unit2   
            corr_matrix = np.corrcoef(Carray_unit.astype(float), Carray_unit2.astype(float))                             
            CaCorrMatrix[indexMapp][indexMapp2]={1: 0.99999, -1: -0.99999}.get(np.round(corr_matrix[0, 1],5), np.round(corr_matrix[0, 1],5))

    df = CaCorrMatrix.apply(pd.to_numeric, errors='coerce')
    ax = sns.heatmap(df, ax=axes[p], square=True, vmin=-.2 , vmax=VMAX , cmap='viridis',
                                    cbar_kws={'shrink': 0.6, 'aspect': 20})
    # Set only the first and last ticks for x and y axes
    # Get the number of ticks
    x_ticks = ax.get_xticks()
    y_ticks = ax.get_yticks()
    ax.set_xticks([x_ticks[0], x_ticks[-1]])
    ax.set_yticks([y_ticks[0], y_ticks[-1]])

    # Optionally, set the labels for these ticks if needed
    ax.set_xticklabels([f'#{int(x_ticks[0])+1}', f'#{int(x_ticks[-1])+1}'],size=6)
    ax.set_yticklabels([f'#{int(y_ticks[0])+1}', f'#{int(y_ticks[-1])+1}'],size=6)

    # Optionally, add labels for the axes if needed
    ax.set_xlabel('L2/3 neurons', labelpad=-2,size=8) if 'L2_3_mice' in str(folder_base) else ax.set_xlabel('L1 neurons', labelpad=-2,size=8)
    ax.set_ylabel('L2/3 neurons', labelpad=-8,size=8) if 'L2_3_mice' in str(folder_base) else ax.set_ylabel('L1 neurons', labelpad=-8,size=8)
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=6)  # Adjust the font size here
    cbar.set_label('Correlation coefficient (r)', fontsize=8, font='Arial', rotation=-90, labelpad=10)
    ax.xaxis.set_ticks_position('top')  # Move ticks to the top
    ax.xaxis.set_label_position('top')  # Move label to the top
    axes[p].set_title(f'{mapp[m]} (r={(np.round(np.nanmean(CaCorrMatrix),2))})', fontsize=12)

plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/Figure2_revised/Extract{mouse}_{folder_base.parts[-2]}_3VigSt_{beg}to{fin}s_2heatmap.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()

Plot raw & filtered LFPs & EMG signals

In [ ]:
CA1t=CA1[round(StartTime*samplerate):round((StartTime + (len(Calcium_sorted.T)/minian_freq))*samplerate)]
PFCt=PFC[round(StartTime*samplerate):round((StartTime + (len(Calcium_sorted.T)/minian_freq))*samplerate)]
S1t=S1[round(StartTime*samplerate):round((StartTime + (len(Calcium_sorted.T)/minian_freq))*samplerate)]
EMGt=EMG[round(StartTime*samplerate):round((StartTime + (len(Calcium_sorted.T)/minian_freq))*samplerate)]

proj_ThetaCWT=filterLFP(CA1,5,9)
proj_Beta1CWT=filterLFP(PFC,10,18)
proj_Beta2CWT=filterLFP(S1,10,18)
proj_DeltaCWT=filterLFP(PFC,1,4)

S1extract=(S1[round(beg*samplerate):round(fin*samplerate)])
BetaS1extract=(proj_Beta2CWT[round(beg*samplerate):round(fin*samplerate)])
DeltaCWTextract=(proj_DeltaCWT[round(beg*samplerate):round(fin*samplerate)])
PFCextract=(PFC[round(beg*samplerate):round(fin*samplerate)])
BetaPFCextract=(proj_Beta1CWT[round(beg*samplerate):round(fin*samplerate)])
CA1extract=(CA1[round(beg*samplerate):round(fin*samplerate)])
Thetaextract=(proj_ThetaCWT[round(beg*samplerate):round(fin*samplerate)])
EMGextract=(EMG[round(beg*samplerate):round(fin*samplerate)])
HypnoExtract=SleepScoredTS_upscaled[round(beg*samplerate):round(fin*samplerate)]


window_size = 10
S1extractDS=S1extract[::100]
BetaS1extractDS=BetaS1extract[::100]
BetaS1extractDSw = uniform_filter1d(BetaS1extractDS, size=window_size, mode='nearest')
DeltaCWTextractDS=DeltaCWTextract[::100]
DeltaCWTextractDSw = uniform_filter1d(DeltaCWTextractDS, size=window_size, mode='nearest')
PFCextractDS=PFCextract[::100]
BetaPFCextractDS=BetaPFCextract[::100]
BetaPFCextractDSw = uniform_filter1d(BetaPFCextractDS, size=window_size, mode='nearest')
CA1extractDS=CA1extract[::100]
ThetaextractDS=Thetaextract[::100]
ThetaextractDSw = uniform_filter1d(ThetaextractDS, size=window_size, mode='nearest')

EMGextractDS=EMGextract[::100]
HypnoExtractDS=HypnoExtract[::100]

#Extract=pd.DataFrame({'S1': S1extract,'filtS1': BetaS1extract,  'PFC': PFCextract, 'filtPFC': BetaPFCextract, 'CA1':CA1extract,'filtCA1': Thetaextract,  'EMG':EMGextract, 'Hypno': HypnoExtract})
#ExtractDS=pd.DataFrame({'S1': S1extractDS, 'filtS1': BetaS1extractDSw, 'filtDelta': DeltaCWTextractDSw, 'PFC': PFCextractDS, 'filtPFC': BetaPFCextractDSw, 'CA1':CA1extractDS,'filtCA1': ThetaextractDSw,  'EMG':EMGextractDS, 'Hypno': HypnoExtractDS})

#CaExtract=zscore(upscaled_Calcium[int(beg/minian_freq*sample_rate):int(fin/minian_freq*sample_rate)]) #DOUBT ON CA_UPSCALED
#CaExtractDS=CaExtract[::100]

UnitName=[f'Unit #{i}' for i in range(len(Calcium))]
#Ca=pd.DataFrame(CaExtract, columns=UnitName)
#CaDSo=pd.DataFrame(CaExtractDS, columns=Calcium.index)
#CaDS = CaDSo*2 + [10 * i for i in range(1, len(CaDSo.columns) + 1)]

#Sum=pd.concat([Extract,Ca], axis=1)
#SumDS=pd.concat([ExtractDS,CaDS], axis=1)


In [ ]:
# Beta power from PFC
plt.close()
plt.figure(figsize=(6, .4))

blend_factor = .5
hex_color = '#2E8B57'
rgb_color = mcolors.hex2color(hex_color)
lighter_color = (1 - blend_factor) * np.array(rgb_color) + blend_factor * np.array([1, 1, 1])
lighter_color_hex = mcolors.to_hex(lighter_color)

plt.plot(zscore(BetaPFCextractDSw)/2, color=lighter_color_hex, linewidth=1.5)
plt.axis('off')  # Hides the axis lines and labels
plt.show()

In [ ]:
plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/Figure1_revised/Extract{mouse}_{folder_base.parts[-2]}_PFCfiltLFPs_{beg}to{fin}s.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True)

In [ ]:
# Beta power from S1
plt.close()
plt.figure(figsize=(6, .4))

blend_factor = .5
hex_color = '#663399'
rgb_color = mcolors.hex2color(hex_color)
lighter_color = (1 - blend_factor) * np.array(rgb_color) + blend_factor * np.array([1, 1, 1])
lighter_color_hex = mcolors.to_hex(lighter_color)

plt.plot(zscore(BetaS1extractDSw)/2, color=lighter_color_hex, linewidth=1.5)
plt.axis('off')  # Hides the axis lines and labels
plt.show()

In [ ]:
plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/Figure1_revised/Extract{mouse}_{folder_base.parts[-2]}_S1filtLFPs_{beg}to{fin}s.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True)

In [ ]:
# All raw and filtered LFPs and EMG
plt.close()
plt.figure(figsize=(6, 2))

blend_factor = .5
hex_color = '#663399'
rgb_color = mcolors.hex2color(hex_color)
lighter_color = (1 - blend_factor) * np.array(rgb_color) + blend_factor * np.array([1, 1, 1])
lighter_color_hex = mcolors.to_hex(lighter_color)

plt.plot(zscore(S1extractDS)/2, color='rebeccapurple', linewidth=.5)
plt.plot(zscore(BetaS1extractDSw)/2, color=lighter_color_hex, linewidth=1)
plt.text(-10, np.mean(zscore(S1extractDS)/2)+.5, 'raw S1', color='rebeccapurple', 
        ha='right', va='center', fontsize=6)  
plt.text(-10, np.mean(zscore(S1extractDS)/2)-.5, 'filt 10-18Hz', color=lighter_color_hex, 
        ha='right', va='center', fontsize=6) 

hex_color = '#2E8B57'
rgb_color = mcolors.hex2color(hex_color)
lighter_color = (1 - blend_factor) * np.array(rgb_color) + blend_factor * np.array([1, 1, 1])
lighter_color_hex = mcolors.to_hex(lighter_color)

plt.plot(zscore(PFCextractDS)/2-3, color='seagreen', linewidth=.5)
plt.plot(zscore(BetaPFCextractDSw)/2-3, color=lighter_color_hex, linewidth=1)
plt.text(-10, np.mean(zscore(PFCextractDS)/2-3)+.5, 'raw PFC', color='seagreen', 
        ha='right', va='center', fontsize=6)  
plt.text(-10, np.mean(zscore(PFCextractDS)/2-3)-.5, 'filt 10-18Hz', color=lighter_color_hex, 
        ha='right', va='center', fontsize=6) 


hex_color = '#4169E1'
rgb_color = mcolors.hex2color(hex_color)
lighter_color = (1 - blend_factor) * np.array(rgb_color) + blend_factor * np.array([1, 1, 1])
lighter_color_hex = mcolors.to_hex(lighter_color)

plt.plot(zscore(CA1extractDS)/2.5-6, color='royalblue', linewidth=.5)
plt.plot(zscore(ThetaextractDSw)/2.5-6, color=lighter_color_hex, linewidth=1)
plt.text(-10, np.mean(zscore(CA1extractDS)/2.5-6)+.5, 'raw CA1', color='royalblue', 
        ha='right', va='center', fontsize=6)  
plt.text(-10, np.mean(zscore(CA1extractDS)/2.5-6)-.5, 'filt 5-9Hz', color=lighter_color_hex, 
        ha='right', va='center', fontsize=6) 


plt.plot(zscore(EMGextractDS)/5-9, color='black', linewidth=.5)
plt.axis('off')  # Hides the axis lines and labels
plt.text(-10, np.mean(zscore(EMGextractDS)/5-9), 'EMG', color='black', 
        ha='right', va='center', fontsize=6)  

plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/Figure1_revised/Extract{mouse}_{folder_base.parts[-2]}_AllLFPs_{beg}to{fin}s.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()


In [ ]:
# All raw and filtered LFPs and EMG
%matplotlib widget
plt.close()
plt.figure(figsize=(6, 2))

blend_factor = .5
hex_color = '#663399'
rgb_color = mcolors.hex2color(hex_color)
lighter_color = (1 - blend_factor) * np.array(rgb_color) + blend_factor * np.array([1, 1, 1])
lighter_color_hex = mcolors.to_hex(lighter_color)

plt.plot((S1extractDS), color='rebeccapurple', linewidth=.5)
plt.text(-10, np.mean(zscore(S1extractDS))+.5, 'raw S1', color='rebeccapurple', 
        ha='right', va='center', fontsize=6)  

hex_color = '#2E8B57'
rgb_color = mcolors.hex2color(hex_color)
lighter_color = (1 - blend_factor) * np.array(rgb_color) + blend_factor * np.array([1, 1, 1])
lighter_color_hex = mcolors.to_hex(lighter_color)

plt.plot((PFCextractDS)-6, color='seagreen', linewidth=.5)
plt.text(-10, np.mean(zscore(PFCextractDS)-6)+.5, 'raw PFC', color='seagreen', 
        ha='right', va='center', fontsize=6)  


hex_color = '#4169E1'
rgb_color = mcolors.hex2color(hex_color)
lighter_color = (1 - blend_factor) * np.array(rgb_color) + blend_factor * np.array([1, 1, 1])
lighter_color_hex = mcolors.to_hex(lighter_color)

plt.plot((CA1extractDS)-12, color='royalblue', linewidth=.5)
plt.text(-10, np.mean(zscore(CA1extractDS)-12)+.5, 'raw CA1', color='royalblue', 
        ha='right', va='center', fontsize=6)  


plt.plot(zscore(((ThetaextractDSw)+2)/((DeltaCWTextractDSw)+2)) -16, color=lighter_color_hex, linewidth=1)
plt.text(-10, np.mean(zscore(EMGextractDS)-16), 'Theta/delta ratio', color=lighter_color_hex, 
        ha='right', va='center', fontsize=6)  

plt.plot((EMGextractDS)-20, color='black', linewidth=.5)
#plt.axis('off')  # Hides the axis lines and labels
plt.text(-10, np.mean(zscore(EMGextractDS)-20), 'EMG', color='black', 
        ha='right', va='center', fontsize=6)  
plt.savefig(f'C:/Users/Manip2/Documents/Manuscripts/Figures_PNASreviews/Figure1_revised/Extract{mouse}_{folder_base.parts[-2]}_AllLFPs_stagg_{beg}to{fin}s.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()
